In [429]:
from queue import Queue
import sys
    
# TRIE 노드 클래스
class TrieNode(object):
    def __init__(self, value, pos=None):
        # 노드의 값 (음절)
        self.value = value
        # 노드의 품사 (비어있지 않으면 형태소로 가능하다는 뜻)
        self.poses = set()
        if pos is not None:
            self.poses.add(pos)
        # 노드의 자식 노드들
        self.children = {}
        # 부모노드
        self.parent = None
        
    """
    노드의 완성형 문자열 계산
    """
    def to_string(self):
        parent_node = self.parent
        string = self.value
        while parent_node is not None and parent_node.value is not None:
            string = parent_node.value + string 
            parent_node = parent_node.parent
        return string
        
# TRIE 구조 클래스        
class Trie(object):
    def __init__(self):
        self.head = TrieNode(None)
    
    """
    사전에 문자열 추가
    """
    def add_word(self, word, pos):
        cur_node = self.head
        
        for char in word:
            if char not in cur_node.children:
                cur_node.children[char] = TrieNode(char)
                cur_node.children[char].value = char
                cur_node.children[char].parent = cur_node
                
            cur_node = cur_node.children[char]
            
        # 형태소 품사 저장
        cur_node.poses.add(pos)
        
    
    """
    문자열이 TRIE에 존재하는지 검색하기
    """
    def search(self, word):
        cur_node = self.head
        
        for char in word:
            if char in cur_node.children:
                cur_node = cur_node.children[char]
            else:
                return None
            
        if len(cur_node.poses) > 0:
            return cur_node
    
    
    """
    prefix로 시작하는 단어를 가진 TRIE 노드들을 검색하기
    """
    def search_prefix(self, prefix):
        cur_node = self.head
        # 결과 노드 리스트
        nodes = []
        # prefix로 시작하는 첫번째 노드
        prefix_node = None
        
        # TRIE에서 prefix로 시작하는 첫번째 노드를 검색하기
        for char in prefix:
            if char in cur_node.children:
                cur_node = cur_node.children[char]
                prefix_node = cur_node
            else:
                return None
            
        # prefix로 시작하는 모든 서브 노드들을 검색하여 nodes에 추가
        queue = list(prefix_node.children.values())
        
        while queue:
            cur = queue.pop()
            if len(cur.poses) > 0:
                nodes.append(cur)
            
            queue += list(curr.children.values())
                
        return nodes
    
    """
    Trie 렌더링
    """
    def render(self, filename):
        from graphviz import Digraph
        i = 0
        diagram = Digraph(comment='Trie')
        diagram.attr('node', fontsize='8')
        diagram.attr('node', height='0.1')
        diagram.attr('node', width='0.1')
        diagram.attr('node', fixedsize='false')
        diagram.attr('node', shape='circle')
        diagram.attr('edge', arrowsize='0.3')
        diagram.node(str(i), self.head.value)

        q = Queue()
        q.put((self.head, i))
        
        # 트리 노드를 순회하며 그래프 생성
        while not q.empty():
            node, parent_index = q.get()

            for value in node.children:
                i = i + 1
                child = node.children[value]
                # 품사가 없다면 끝맺음 노드가 될 수 없음
                if len(child.poses) == 0:
                    diagram.attr('node', shape='diamond')
                    diagram.node(str(i), child.value)
                else:
                    # 끝나는 노드는 원형으로 표시
                    diagram.attr('node', shape='box')
                    diagram.attr('node', fontname='NanumGothic')
                    diagram.node(str(i), child.value + '/' + ",".join(list(child.poses)))
                diagram.edge(str(parent_index), str(i))
                q.put((child, i))
                
        diagram.render(filename + '.gv', view=False)     
        

In [383]:
# 좌우 접속정보를 담고 있기 위한 노드
class PosConnectionNode:
    def __init__(self, value):
        self.value = value
        # 좌측 접속가능 품사연결 노드
        self.left_poses = {}
        # 우측 접속가능 품사연결 노드
        self.right_poses = {}
        
    """
    좌측접속가능품사 확인
    """
    def left_has(self, left_pos):
        return left_pos in self.left_poses
    
    """
    우측접속가능품사 확인
    """
    def right_has(self, right_pos):
        return right_pos in self.right_poses
    
    """
    좌측접속가능품사 추가
    """
    def add_left(self, left_pos):
        self.left_poses[left_pos] = PosConnectionNode(left_pos)
       
    """
    우측접속가능품사 추가
    """
    def add_right(self, right_pos):
        self.right_poses[right_pos] = PosConnectionNode(right_pos)

# 좌우 접속정보 규칙 추가
def add_connection_rule(left_pos, right_pos):
    global pos_connection_table
    if left_pos not in pos_connection_table:
        pos_connection_table[left_pos] = PosConnectionNode(left_pos)
    left_pos_node = pos_connection_table[left_pos]
    
    if right_pos not in pos_connection_table:
        pos_connection_table[right_pos] = PosConnectionNode(right_pos)
    right_pos_node = pos_connection_table[right_pos]
    
    if not left_pos_node.right_has(right_pos):
        left_pos_node.add_right(right_pos)
        right_pos_node.add_left(left_pos)
    
# 좌우 접속정보 가능한지 확인
def check_connection_rule(left_pos, right_pos):
    if left_pos not in pos_connection_table:
        return False
    left_pos_node = pos_connection_table[left_pos]
    return left_pos_node.right_has(right_pos)

In [384]:
# Tabular Parsing을 위한 노드
class TabularNode(object):
    def __init__(self):
        # 가능한 형태소조합들 (TrieNode[][])
        self.trie_nodes_set = []
        

In [435]:
# 주어진 사전을 사용하여 어절 형태소를 Tabular parsing한다.
def tabular_parsing(trie, sentence):
    size = len(sentence)
    
    # Dynamic Programming
    table = [[0] * l for l in range(size, 0, -1)]
    
    # Bottom Up filling
    for j in range(0, size):
        for i in range(j, -1, -1):
            table[i][j - i] = TabularNode()
            cur_tabular_node = table[i][j - i]
            cur_morph = trie.search(sentence[i:j+1])
            # 현재 칸에 해당하는 값을 Trie 사전에서 조회하여 형태소 채우기
            if cur_morph is not None:
                cur_tabular_node.trie_nodes_set.append([cur_morph])

            # 쪼개지지 않는 경우
            if i == j:
                continue

            # 쪼개지는 경우 T(i, k) + T(k+1, j) 모든 경우 Bottom Up 순회
            for k in range(i, j):
                # T(i,k) + T(k+1,j) 좌우 노드 조회
                left_tabular_node = table[i][k - i]
                right_tabular_node = table[k - i + 1][j - k - 1]
                    
                # 현재 선택된 좌우 형태소의 각 품사가 여러개일 수 있으므로, 가능한 모든 품사 조합을 확인한다.
                for left_trie_nodes in left_tabular_node.trie_nodes_set:
                    for left_trie_pos in left_trie_nodes[-1].poses:
                        for right_trie_nodes in right_tabular_node.trie_nodes_set:
                            for right_trie_pos in right_trie_nodes[0].poses:
                                # 좌우접속정보 확인
                                if check_connection_rule(left_trie_pos, right_trie_pos) is True:
                                    # 접속 가능한 연결에 대해 품사 확정하여 신규 노드 생성
                                    tmp_left_trie = TrieNode(left_trie_nodes[-1].value)
                                    tmp_left_trie.poses = set([left_trie_pos])
                                    tmp_left_trie.parent = left_trie_nodes[-1].parent
                                    left_trie_nodes[-1] = tmp_left_trie
                                    
                                    tmp_right_trie = TrieNode(right_trie_nodes[0].value)
                                    tmp_right_trie.poses = set([right_trie_pos])
                                    tmp_right_trie.parent = right_trie_nodes[0].parent
                                    right_trie_nodes[0] = tmp_right_trie
                                    
                                    # 현재 테이블 칸에 접속 가능 케이스 추가
                                    cur_tabular_node.trie_nodes_set.append(left_trie_nodes + right_trie_nodes)
                            
                
    # 최종으로 채워진 Table 리턴
    return table

# 문법 파일에서 무시할 행 체크
def ispass(line):
    return line.startswith("#") or not line or line.isspace() or (len(line) == 1 and line == '\r')

# Tabular Parsing 결과 테이블의 특정 칸 정보 출력
def print_tabular_node(node):
    print('[', end='')
    for i, trie_nodes in enumerate(node.trie_nodes_set):
        
        for j, trie_node in enumerate(trie_nodes):
            print(trie_node.to_string() + '/' + list(trie_node.poses)[0], end='')
            if j < len(trie_nodes) - 1:
                print('+', end='')
        if i < len(node.trie_nodes_set) - 1:
            print(', ', end='')
    print(']')
        

# 좌우접속정보 표
pos_connection_table = {}

# Trie 생성
trie = Trie()

# 사전 및 좌우접속정보 데이터 해석
o = open('grammar.txt', 'r', encoding='utf-8')
for line in o.readlines():
    if ispass(line):
        continue
    word, morphs = line.split('\t')
    morph_pos_tuples = [morph_pos.strip().split('/') for morph_pos in [morph_pos for morph_pos in morphs.split('+')]]
    
    # 형태소/품사 쌍 처리
    for i in range(0, len(morph_pos_tuples)):
        morph_pos = morph_pos_tuples[i]
        # Trie 사전에 엔트리/품사 추가
        trie.add_word(morph_pos[0], morph_pos[1])
        
        # 좌우접속정보 규칙 추가
        if i < len(morph_pos_tuples) - 1:
            add_connection_rule(morph_pos[1], morph_pos_tuples[i + 1][1])
o.close()

# 사전 PDF로 렌더링
trie.render("trie")

sentences = []
if len(sys.argv) == 2:
    sentences.append(sys.argv[1])
else:
    # 마지막 문장은 기존에 없던 예문 !!
    sentences.extend(['환절기에는 일교차가 심하니 특히 감기를 조심하자.',
                 '나는 종원이가 박사를 하기로 마음을 먹었다고 들었다.',
                 '사실 명언이기 위해서는 그 사람이 우리에게 하는 말이 머릿속에 오래 기억되고 다시 생각 나야 한다.',
                 '문제를 해결하는 능력도 중요하지만 새로운 문제를 발견하고 왜 문제인지 이해하는 것이 우선인 것을 명심하자.',
                 '좋아하는 것은 그 대상이 나를 기쁘게 해주기를 바라는 것이고 사랑은 나로 인해 그 대상이 기쁘기를 바라는 것이다.',
                 '나는 머릿속에 사랑하는 대상을 발견하고 기억하고 다시 생각을 한다.'])
                      
for sentence in sentences:
    print('"', sentence, '"', ' 분석 결과')
    for word in sentence.split(' '):
        parsed_node = tabular_parsing(trie, word)
        print_tabular_node(parsed_node[0][len(parsed_node[0]) - 1])
    print()
    

" 환절기에는 일교차가 심하니 특히 감기를 조심하자. "  분석 결과
[환절기/보통명사+에/부사격조사+는/보조사]
[일교차/보통명사+가/주격조사]
[심하/형용사+니/의존적연결어미]
[특히/일반부사]
[감기/보통명사+를/목적격조사]
[조심/보통명사+하/동사파생접미사+자/종결어미+./마침표]

" 나는 종원이가 박사를 하기로 마음을 먹었다고 들었다. "  분석 결과
[나/대명사+는/보조사]
[종원이/보통명사+가/주격조사]
[박사/보통명사+를/목적격조사]
[하기로/동사, 하/동사+기/명사형전성어미+로/부사격조사]
[마음/보통명사+을/목적격조사]
[먹/동사+었/선어말어미+다고/연결어미]
[들/보조용언+었/선어말어미+다/종결어미+./마침표]

" 사실 명언이기 위해서는 그 사람이 우리에게 하는 말이 머릿속에 오래 기억되고 다시 생각 나야 한다. "  분석 결과
[사실/일반부사]
[명언/보통명사+이/긍정지정사+기/명사형전성어미]
[위해/동사+서/의존적연결어미+는/보조사]
[그/일반관형사]
[사람/보통명사+이/긍정지정사, 사람/보통명사+이/주격조사]
[우리/대명사+에게/부사격조사]
[하/동사파생접미사+는/관형형전성어미, 하/동사+는/관형형전성어미]
[말/보통명사+이/긍정지정사, 말/보통명사+이/주격조사]
[머릿속/보통명사+에/부사격조사]
[오래/일반부사]
[기억/보통명사+되/동사파생접미사+고/대등연결어미, 기억/보통명사+되/동사파생접미사+고/의존적연결어미]
[다시/일반부사]
[생각/보통명사]
[나/동사+야/의존적연결어미]
[한다/동사+./마침표]

" 문제를 해결하는 능력도 중요하지만 새로운 문제를 발견하고 왜 문제인지 이해하는 것이 우선인 것을 명심하자. "  분석 결과
[문제/보통명사+를/목적격조사]
[해결/보통명사+하/동사파생접미사+는/관형형전성어미]
[능력/보통명사+도/보조사]
[중요/보통명사+하/동사파생접미사+지만/대등연결어미]
[새로운/형용사]
[문제/보통명사+를/목적격조사]
[발견/보통명사+하/동사파생접미사+고/대등연결어미, 발견/보통명사+하/동사파생접미사+고